In [2]:
%pip install numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.datasets import load_iris, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd

In [ ]:
X,y = fetch_openml(name='adult', version=2, as_frame=True, return_X_y=True)

In [ ]:
X[1:5]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States


In [6]:
# import numpy as np

# np.random.seed(42)
# n_amostras = 10000

# # Colunas numéricas
# idades = np.random.randint(18, 70, size=n_amostras)
# salarios = np.random.randint(1500, 15000, size=n_amostras)

# # Colunas categóricas
# generos = np.random.choice(['M', 'F'], size=n_amostras)
# estados_civis = np.random.choice(['solteiro', 'casado', 'divorciado'], size=n_amostras)

# # Gerando risco baseado em regras lógicas (não aleatório)
# riscos = []
# for i in range(n_amostras):
#     idade = idades[i]
#     salario = salarios[i]
#     estado = estados_civis[i]
    
#     if salario < 4000:
#         risco = 'alto'
#     elif salario < 8000:
#         risco = 'medio'
#     else:
#         risco = 'baixo'
#     riscos.append(risco)

# riscos = np.array(riscos)

# # Combinar tudo
# dados = np.column_stack((idades, estados_civis, salarios, generos, riscos))

# # Separar features e target
# X = dados[:, :-1]
# y = dados[:, -1]

# print("Exemplo de 5 amostras:")
# print(dados[:5])
# print("\nDistribuição de riscos:", np.unique(y, return_counts=True))


In [7]:
y

0        <=50K
1        <=50K
2         >50K
3         >50K
4        <=50K
         ...  
48837    <=50K
48838     >50K
48839    <=50K
48840    <=50K
48841     >50K
Name: class, Length: 48842, dtype: category
Categories (2, object): ['<=50K', '>50K']

In [8]:
X,y

(       age     workclass  fnlwgt     education  education-num  \
 0       25       Private  226802          11th              7   
 1       38       Private   89814       HS-grad              9   
 2       28     Local-gov  336951    Assoc-acdm             12   
 3       44       Private  160323  Some-college             10   
 4       18           NaN  103497  Some-college             10   
 ...    ...           ...     ...           ...            ...   
 48837   27       Private  257302    Assoc-acdm             12   
 48838   40       Private  154374       HS-grad              9   
 48839   58       Private  151910       HS-grad              9   
 48840   22       Private  201490       HS-grad              9   
 48841   52  Self-emp-inc  287927       HS-grad              9   
 
            marital-status         occupation relationship   race     sex  \
 0           Never-married  Machine-op-inspct    Own-child  Black    Male   
 1      Married-civ-spouse    Farming-fishing      H

In [9]:
X,y

for x,y_result in zip(X,y):
    print(x,'->',y_result)

age -> <=50K
workclass -> <=50K
fnlwgt -> >50K
education -> >50K
education-num -> <=50K
marital-status -> <=50K
occupation -> <=50K
relationship -> >50K
race -> <=50K
sex -> <=50K
capital-gain -> >50K
capital-loss -> <=50K
hours-per-week -> <=50K
native-country -> <=50K


In [10]:
class naiveBayes():
    
    def __init__(self):
        self.mean = []
        self.std = []
        self.probs = []
        self.probs_categoric = []
        self.classes = []
    
    # Função para detectar colunas numéricas e categóricas
    def separar_colunas(self,dados):
        colunas_numericas = []
        colunas_categoricas = []

        for i in range(dados.shape[1]):
            coluna = dados[:, i]
            # Tenta converter para float — se não der, é categórica
            try:
                coluna.astype(float)
                colunas_numericas.append(i)
            except ValueError:
                colunas_categoricas.append(i)
        
        return colunas_numericas, colunas_categoricas
    
    def fit_categoric(self, X, y):
        _, categoricas = self.separar_colunas(X)
        self.probs_categoric = []

        for x in categoricas:
            valores_possiveis = np.unique(X[:, x])
            probs_atributo = []

            for i, classe in enumerate(self.classes[0]):
                dados_classe = X[y == classe, x]
                vals, counts = np.unique(dados_classe, return_counts=True)
                
                prob_dict = dict(zip(vals, counts / self.classes[1][i]))

                probs = [prob_dict.get(v, 0) for v in valores_possiveis]
                probs_atributo.append(probs)

            self.probs_categoric.append((x, valores_possiveis, probs_atributo))
    
    def fit(self,X,y):
        self.classes = np.unique(y,return_counts=True)
        # print('Classes encontradas:',self.classes)
        for classe in self.classes[0]:
            numericas,_ = self.separar_colunas(X)
            
            counts_data = X[y==classe]
            counts_data_numerics = counts_data[:,numericas].astype(float)
            
            self.mean.append(np.mean(counts_data_numerics))
            self.std.append(np.std(counts_data_numerics))
            
            self.probs.append(len(counts_data_numerics) / len(X))
                
        self.fit_categoric(X, y)
        
    def predict(self, X):
        preds = []

        # identificar tipos de colunas
        numericas, categoricas = self.separar_colunas(X)

        for amostra in X:
            probs_classes = []

            # percorre cada classe (ex: baixo, medio, alto)
            for i, _ in enumerate(self.classes[0]):
                # --- Parte 1: probabilidade base da classe (P(classe))
                prob_total = self.probs[i]

                # --- Parte 2: features numéricas (Gaussiana)
                if numericas:
                    x_num = amostra[numericas].astype(float)
                    prob_gauss = np.prod(self.gauss(self.mean[i], self.std[i], x_num))
                    prob_total *= prob_gauss

                # --- Parte 3: features categóricas
                for col_idx, valores, probs_por_classe in self.probs_categoric:
                    valor = amostra[col_idx]
                    if valor in valores:
                        pos = np.where(valores == valor)[0][0]
                        prob_total *= probs_por_classe[i][pos]
                    else:
                        prob_total *= 1e-6  # suavização: valor não visto

                probs_classes.append(prob_total)

            # normaliza probabilidades (para interpretar como confiança)
            probs_classes = np.array(probs_classes)
            probs_classes_norm = probs_classes / probs_classes.sum()

            # classe com maior probabilidade
            pred_classe = self.classes[0][np.argmax(probs_classes)]
            preds.append((pred_classe, probs_classes_norm))

        return preds
    
    def gauss(self,mean,std,x):
        return (1 / (np.sqrt(2 * np.pi) * std)) * np.e ** (- (x - mean) ** 2 / (2 * std ** 2))

In [11]:
nB = naiveBayes()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42, shuffle=True)

for x,y_result in zip(X_train,y_train):
    print(x,'->',y_result)

age -> <=50K
workclass -> <=50K
fnlwgt -> <=50K
education -> <=50K
education-num -> <=50K
marital-status -> <=50K
occupation -> <=50K
relationship -> <=50K
race -> <=50K
sex -> <=50K
capital-gain -> <=50K
capital-loss -> <=50K
hours-per-week -> <=50K
native-country -> <=50K


In [13]:
nB.separar_colunas(X_train)

InvalidIndexError: (slice(None, None, None), 0)

In [ ]:
nB.fit(X_train,y_train)

In [ ]:
nB.probs_categoric

In [ ]:
for atributo in nB.probs_categoric:
    print(atributo)
    # for item in atributo:
    #     print(item)

In [ ]:
nB.mean, nB.std, nB.probs,nB.probs_categoric,nB.classes

In [ ]:
nB.probs

In [ ]:
nB.probs_categoric

In [ ]:
for atributo, valores, probs_por_classe in nB.probs_categoric:
    print(f"Atributo coluna {atributo} com valores {valores}")
    for i, classe in enumerate(nB.classes[0]):
        print(f"  Classe '{classe}': Probabilidades {probs_por_classe[i]}")
        if sum(probs_por_classe[i]) != 1:
            print("    (A soma das probabilidades não é 1!)")
        else:
            print("    (A soma das probabilidades é 1!)")

In [ ]:
y_pred = nB.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred = [pred[0] for pred in y_pred]

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)